In [2]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    # This catches raw numbers that look like Aadhaar numbers
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    # This catches Aadhaar numbers that are labeled
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    # Use with caution as it might pick up other 12-digit numbers
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None

def clean_extracted_field(text, field_type):
    """
    Cleans extracted text based on field type to remove common OCR artifacts
    and mislabeled content
    """
    # Convert to string in case we received another type
    text = str(text).strip()
    
    # Remove common label text that might be captured within the value
    unwanted_labels = [
        "Phone Number", "Contact", "Mobile", "Call",
        "Hospital Name", "Doctor", "Clinic", "MD", "Dr\\.",
        "Address", "Location", "Place", "Residence",
        "Insurance ID", "Policy Number", "Insurance",
        "Amount", "Total", "Fee", "Payment",
        "Disease", "Diagnosis", "Condition",
        "Medicines", "Medication", "Drugs", "Prescription"
    ]
    
    # For each unwanted label, try to remove it if it appears at the end
    for label in unwanted_labels:
        # Create pattern to match label at the end of the text (allowing for spaces)
        pattern = rf'\s*{re.escape(label)}$'
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove common field separators
    text = re.sub(r'[:;|]$', '', text)
    
    # Clean up newlines and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Additional field-specific cleaning
    if field_type in ["Address"]:
        # Keep only relevant address information
        text = re.sub(r'\s*(?:Phone|Mobile|Contact|Email).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Hospital Name"]:
        # Remove doctor references
        text = re.sub(r'\s*(?:Doctor|Dr\.|MD|Physician).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Phone Number"]:
        # Keep only digits and basic formatting characters
        text = re.sub(r'[^\d+\-\s()]', '', text)
    
    return text.strip()

def extract_fields_with_boundaries(text):
    """
    Extract fields with improved boundary detection to prevent label bleed
    """
    extracted_info = []
    found_labels = set()
    
    # Dictionary of field patterns with better boundary detection
    field_patterns = {
        "Name": r'(?:Patient(?:\s*Name)?|Name|Patient)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Father|Gender|Blood|Aadhaar))',
        "Father's Name": r'(?:Father(?:[\'s]*\s*Name)?|Father)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Gender|Blood|Aadhaar))',
        "Gender": r'(?:Gender|Sex)[:;]?\s*(Male|Female|Other|M|F)(?=\n|$)',
        "Blood Group": r'(?:Blood(?:\s*Group)?)[:;]?\s*([ABO][+-]|AB[+-])(?=\n|$)',
        "Address": r'(?:Address|Location|Place|Residence)[:;]?\s*([\w\s,\.\-\/]+?)(?=\n|$|(?:Phone|Mobile|Contact|Email))',
        "Hospital Name": r'(?:Hospital(?:\s*Name)?|Clinic|Medical Center)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Doctor|Dr|MD|Address))',
        "Insurance ID": r'(?:Insurance(?:\s*(?:ID|Number|No))?|Policy(?:\s*Number)?)[:;]?\s*([\w\d\-]+?)(?=\n|$)',
        "Phone Number": r'(?:Phone(?:\s*Number)?|Mobile|Contact|Cell)[:;]?\s*([\d\s\+\-\(\)]+?)(?=\n|$)',
        "Amount": r'(?:Amount|Total|Cost|Fee|Charges)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)',
        "Disease Name": r'(?:Disease(?:\s*Name)?|Diagnosis|Condition|Ailment)[:;]?\s*([\w\s]+?)(?=\n|$|(?:Disease Details|Symptoms|Treatment))',
        "Disease Details": r'(?:Disease(?:\s*Details)?|Details|Diagnosis Details|Clinical Details|Symptoms)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Medicines|Medications|Drugs)|$)',
        "Medicines": r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Bed|Ventilation|Amount|Charges)|$)',
        "Bed Type": r'(?:Bed(?:\s*Type)?)[:;]?\s*([\w\s]+?)(?=\n|$)',
        "Ventilation": r'(?:Ventilation|Ventilator|Oxygen)[:;]?\s*(Yes|No|Required|Not Required)(?=\n|$)',
        "Other Charges": r'(?:Other(?:\s*Charges)?|Additional(?:\s*Charges)?|Extra)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)'
    }
    
    # 1. First pass: Extract Aadhaar number with dedicated function
    aadhaar = extract_aadhaar_number(text)
    if aadhaar:
        formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
        extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})
        found_labels.add("Aadhar Card")
    
    # 2. Second pass: Extract other fields with improved boundary detection
    for label, pattern in field_patterns.items():
        if label in found_labels:
            continue
            
        matches = re.search(pattern, text, re.IGNORECASE)
        if matches:
            extracted_text = matches.group(1).strip()
            # Clean the extracted text to remove potential label contamination
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            # Only add if we have meaningful content
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    # 3. Third pass: Look for unlabeled numbers that might be specific fields
    if "Phone Number" not in found_labels:
        # Look for potential phone numbers (10-digit sequences)
        phone_matches = re.search(r'(?<!\d)(\d{10})(?!\d)', text)
        if phone_matches:
            extracted_info.append({"Text": phone_matches.group(1), "Label": "Phone Number"})
            found_labels.add("Phone Number")
    
    # Look for Appendicitis or other common conditions if disease name not found
    if "Disease Name" not in found_labels:
        common_diseases = ["appendicitis", "diabetes", "hypertension", "cancer", "fracture", "pneumonia"]
        for disease in common_diseases:
            if re.search(rf'\b{disease}\b', text, re.IGNORECASE):
                extracted_info.append({"Text": disease.capitalize(), "Label": "Disease Name"})
                found_labels.add("Disease Name")
                break
    
    return extracted_info

def process_text(text, keywords=[]):
    """
    Main processing function that combines extraction methods
    """
    # Get fields using improved boundary detection
    extracted_info = extract_fields_with_boundaries(text)
    
    # For backward compatibility, still use keyword-based extraction for any missing fields
    found_labels = {item["Label"] for item in extracted_info}
    
    for keyword in keywords:
        # Skip keywords for fields we already found
        label = keyword.replace(":", "").strip()
        if any(label in existing for existing in found_labels):
            continue
            
        # Simple keyword-based extraction as fallback
        pattern = re.compile(rf"{re.escape(keyword)}\s*([\w\s\d\.\-]+?)(?=\n|$)", re.IGNORECASE)
        match = pattern.search(text)
        if match:
            extracted_text = match.group(1).strip()
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    return extracted_info

# The extract_aadhaar_number function from previous solution should be included here
def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None
def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Modified query to exclude the ventilation and appointment_time columns
        insert_patient_query = """
        INSERT INTO patient_details 
        (insurance_id, name, father_name, aadhar_card, gender, blood_group, 
        address, hospital_name, phone_number, amount, 
        disease_name, disease_details, medicines, bed_type, other_charges) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        
        # Clean the amount value
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        if amount:
            amount = re.sub(r'[^\d.]', '', amount)
            if amount:
                try:
                    amount = float(amount)
                except ValueError:
                    amount = None
        
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        
        # Clean other_charges
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)
        if other_charges:
            other_charges = re.sub(r'[^\d.]', '', other_charges)
            if other_charges:
                try:
                    other_charges = float(other_charges)
                except ValueError:
                    other_charges = None

        # Note: ventilation and appointment_time are removed from the values tuple
        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, 
                          address, hospital_name, phone_number, amount, 
                          disease_name, disease_details, medicines, bed_type, other_charges)
        
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")
def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/HealAI/HealAI', filename='', title='', show_hidden=False, sel…

Output()

MySQl ------> MongoDB

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
from pymongo import MongoClient
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MongoDB connection details
MONGO_CONFIG = {
    "host": "127.0.0.1",
    "port": 27017,
    "db_name": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    # This catches raw numbers that look like Aadhaar numbers
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    # This catches Aadhaar numbers that are labeled
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    # Use with caution as it might pick up other 12-digit numbers
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None

def clean_extracted_field(text, field_type):
    """
    Cleans extracted text based on field type to remove common OCR artifacts
    and mislabeled content
    """
    # Convert to string in case we received another type
    text = str(text).strip()
    
    # Remove common label text that might be captured within the value
    unwanted_labels = [
        "Phone Number", "Contact", "Mobile", "Call",
        "Hospital Name", "Doctor", "Clinic", "MD", "Dr\\.",
        "Address", "Location", "Place", "Residence",
        "Insurance ID", "Policy Number", "Insurance",
        "Amount", "Total", "Fee", "Payment",
        "Disease", "Diagnosis", "Condition",
        "Medicines", "Medication", "Drugs", "Prescription"
    ]
    
    # For each unwanted label, try to remove it if it appears at the end
    for label in unwanted_labels:
        # Create pattern to match label at the end of the text (allowing for spaces)
        pattern = rf'\s*{re.escape(label)}$'
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove common field separators
    text = re.sub(r'[:;|]$', '', text)
    
    # Clean up newlines and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Additional field-specific cleaning
    if field_type in ["Address"]:
        # Keep only relevant address information
        text = re.sub(r'\s*(?:Phone|Mobile|Contact|Email).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Hospital Name"]:
        # Remove doctor references
        text = re.sub(r'\s*(?:Doctor|Dr\.|MD|Physician).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Phone Number"]:
        # Keep only digits and basic formatting characters
        text = re.sub(r'[^\d+\-\s()]', '', text)
    
    return text.strip()

def extract_fields_with_boundaries(text):
    """
    Extract fields with improved boundary detection to prevent label bleed
    """
    extracted_info = []
    found_labels = set()
    
    # Dictionary of field patterns with better boundary detection
    field_patterns = {
        "Name": r'(?:Patient(?:\s*Name)?|Name|Patient)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Father|Gender|Blood|Aadhaar))',
        "Father's Name": r'(?:Father(?:[\'s]*\s*Name)?|Father)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Gender|Blood|Aadhaar))',
        "Gender": r'(?:Gender|Sex)[:;]?\s*(Male|Female|Other|M|F)(?=\n|$)',
        "Blood Group": r'(?:Blood(?:\s*Group)?)[:;]?\s*([ABO][+-]|AB[+-])(?=\n|$)',
        "Address": r'(?:Address|Location|Place|Residence)[:;]?\s*([\w\s,\.\-\/]+?)(?=\n|$|(?:Phone|Mobile|Contact|Email))',
        "Hospital Name": r'(?:Hospital(?:\s*Name)?|Clinic|Medical Center)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Doctor|Dr|MD|Address))',
        "Insurance ID": r'(?:Insurance(?:\s*(?:ID|Number|No))?|Policy(?:\s*Number)?)[:;]?\s*([\w\d\-]+?)(?=\n|$)',
        "Phone Number": r'(?:Phone(?:\s*Number)?|Mobile|Contact|Cell)[:;]?\s*([\d\s\+\-\(\)]+?)(?=\n|$)',
        "Amount": r'(?:Amount|Total|Cost|Fee|Charges)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)',
        "Disease Name": r'(?:Disease(?:\s*Name)?|Diagnosis|Condition|Ailment)[:;]?\s*([\w\s]+?)(?=\n|$|(?:Disease Details|Symptoms|Treatment))',
        "Disease Details": r'(?:Disease(?:\s*Details)?|Details|Diagnosis Details|Clinical Details|Symptoms)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Medicines|Medications|Drugs)|$)',
        "Medicines": r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Bed|Ventilation|Amount|Charges)|$)',
        "Bed Type": r'(?:Bed(?:\s*Type)?)[:;]?\s*([\w\s]+?)(?=\n|$)',
        "Ventilation": r'(?:Ventilation|Ventilator|Oxygen)[:;]?\s*(Yes|No|Required|Not Required)(?=\n|$)',
        "Other Charges": r'(?:Other(?:\s*Charges)?|Additional(?:\s*Charges)?|Extra)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)'
    }
    
    # 1. First pass: Extract Aadhaar number with dedicated function
    aadhaar = extract_aadhaar_number(text)
    if aadhaar:
        formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
        extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})
        found_labels.add("Aadhar Card")
    
    # 2. Second pass: Extract other fields with improved boundary detection
    for label, pattern in field_patterns.items():
        if label in found_labels:
            continue
            
        matches = re.search(pattern, text, re.IGNORECASE)
        if matches:
            extracted_text = matches.group(1).strip()
            # Clean the extracted text to remove potential label contamination
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            # Only add if we have meaningful content
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    # 3. Third pass: Look for unlabeled numbers that might be specific fields
    if "Phone Number" not in found_labels:
        # Look for potential phone numbers (10-digit sequences)
        phone_matches = re.search(r'(?<!\d)(\d{10})(?!\d)', text)
        if phone_matches:
            extracted_info.append({"Text": phone_matches.group(1), "Label": "Phone Number"})
            found_labels.add("Phone Number")
    
    # Look for Appendicitis or other common conditions if disease name not found
    if "Disease Name" not in found_labels:
        common_diseases = ["appendicitis", "diabetes", "hypertension", "cancer", "fracture", "pneumonia"]
        for disease in common_diseases:
            if re.search(rf'\b{disease}\b', text, re.IGNORECASE):
                extracted_info.append({"Text": disease.capitalize(), "Label": "Disease Name"})
                found_labels.add("Disease Name")
                break
    
    return extracted_info

def process_text(text, keywords=[]):
    """
    Main processing function that combines extraction methods
    """
    # Get fields using improved boundary detection
    extracted_info = extract_fields_with_boundaries(text)
    
    # For backward compatibility, still use keyword-based extraction for any missing fields
    found_labels = {item["Label"] for item in extracted_info}
    
    for keyword in keywords:
        # Skip keywords for fields we already found
        label = keyword.replace(":", "").strip()
        if any(label in existing for existing in found_labels):
            continue
            
        # Simple keyword-based extraction as fallback
        pattern = re.compile(rf"{re.escape(keyword)}\s*([\w\s\d\.\-]+?)(?=\n|$)", re.IGNORECASE)
        match = pattern.search(text)
        if match:
            extracted_text = match.group(1).strip()
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        # Connect to MongoDB
        client = MongoClient(MONGO_CONFIG["host"], MONGO_CONFIG["port"])
        db = client[MONGO_CONFIG["db_name"]]
        
        # Create document collections
        patient_documents = db.patient_documents
        patient_details = db.patient_details

        # Insert document information
        doc_data = {
            "insurance_id": insurance_id,
            "file_path": file_path,
            "created_at": datetime.now()
        }
        
        doc_result = patient_documents.insert_one(doc_data)
        
        # Create patient details document
        patient_data = {
            "insurance_id": insurance_id,
            "name": next((item["Text"] for item in data if item["Label"] == "Name"), None),
            "father_name": next((item["Text"] for item in data if item["Label"] == "Father's Name"), None),
            "aadhar_card": next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None),
            "gender": next((item["Text"] for item in data if item["Label"] == "Gender"), None),
            "blood_group": next((item["Text"] for item in data if item["Label"] == "Blood Group"), None),
            "address": next((item["Text"] for item in data if item["Label"] == "Address"), None),
            "hospital_name": next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None),
            "phone_number": next((item["Text"] for item in data if item["Label"] == "Phone Number"), None),
            "disease_name": next((item["Text"] for item in data if item["Label"] == "Disease Name"), None),
            "disease_details": next((item["Text"] for item in data if item["Label"] == "Disease Details"), None),
            "medicines": next((item["Text"] for item in data if item["Label"] == "Medicines"), None),
            "bed_type": next((item["Text"] for item in data if item["Label"] == "Bed Type"), None),
            "created_at": datetime.now()
        }
        
        # Handle numeric fields
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        if amount:
            amount = re.sub(r'[^\d.]', '', amount)
            if amount:
                try:
                    patient_data["amount"] = float(amount)
                except ValueError:
                    patient_data["amount"] = None
                    
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)
        if other_charges:
            other_charges = re.sub(r'[^\d.]', '', other_charges)
            if other_charges:
                try:
                    patient_data["other_charges"] = float(other_charges)
                except ValueError:
                    patient_data["other_charges"] = None
        
        # Insert patient details
        patient_result = patient_details.insert_one(patient_data)
        
        with output_widget:
            print("Data saved to MongoDB successfully.")
            print(f"Document ID: {doc_result.inserted_id}")
            print(f"Patient Details ID: {patient_result.inserted_id}")
            
    except Exception as err:
        with output_widget:
            print(f"Error saving to MongoDB: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/HealAI/HealAI', filename='', title='', show_hidden=False, sel…

Output()

Trying to implement gemini api:

In [ ]:
from typing import Dict, List, Any
import re
import pymongo
import google.generativeai as genai
import creds

class PatientChatbot:
    def __init__(self, mongodb_uri: str, gemini_api_key: str):
        """Initialize chatbot with MongoDB and Gemini API connections"""
        # Connect to MongoDB
        self.mongo_client = pymongo.MongoClient(mongodb_uri)
        self.db = self.mongo_client["healthcare_docs"]
        
        # Initialize Gemini API
        genai.configure(api_key=gemini_api_key)
        self.model = genai.GenerativeModel('gemini-2.0-flash')
        
        # Define intent patterns
        self.intents = {
            "claim_status": re.compile(r'(status|update).*(claim|application)'),
            "charge_explanation": re.compile(r'(what|explain|clarify).*(charge|fee|cost|bill)'),
            "document_required": re.compile(r'(what|which).*(document|paper|proof).*(need|require)'),
            "coverage_question": re.compile(r'(cover|include|reimburse).*(policy|insurance)'),
            "greeting": re.compile(r'(hello|hi|hey|greetings)')
        }
        
    def _fetch_patient_data(self, patient_id: str) -> Dict:
        """Fetch patient data from MongoDB"""
        patient_data = self.db.patients.find_one({"_id": patient_id})
        if not patient_data:
            return {}
            
        # Also fetch related claims
        claims = list(self.db.claims.find({"patient_id": patient_id}))
        patient_data["claims"] = claims
        
        return patient_data
        
    def _detect_intent(self, query: str) -> str:
        """Detect the intent of user query"""
        for intent, pattern in self.intents.items():
            if pattern.search(query.lower()):
                return intent
        return "general_query"
        
    def _extract_entities(self, query: str) -> Dict:
        """Extract entities from user query"""
        entities = {}
        
        # Extract claim ID if present
        claim_match = re.search(r'claim\s+(?:id\s+)?([A-Z0-9]+)', query, re.IGNORECASE)
        if claim_match:
            entities["claim_id"] = claim_match.group(1)
        
        # Extract other entities as needed
        
        return entities
        
    def _generate_response(self, intent: str, entities: Dict, patient_data: Dict, query: str) -> str:
        """Generate response based on intent, entities and patient data"""
        context = self._create_context(intent, entities, patient_data)
        
        prompt = f"""
        You are a helpful healthcare assistant. Answer the patient's question using the context provided.
        
        CONTEXT:
        {context}
        
        PATIENT QUESTION:
        {query}
        
        INSTRUCTIONS:
        - Answer in simple, patient-friendly language
        - Explain any medical or insurance terms
        - Keep answers concise (3-4 sentences max)
        - If information is missing from context, admit you don't know
        """
        
        response = self.model.generate_content(prompt)
        return response.text
        
    def _create_context(self, intent: str, entities: Dict, patient_data: Dict) -> str:
        """Create context for Gemini based on intent and available data"""
        context = ""
        
        if intent == "claim_status" and patient_data.get("claims"):
            claim_id = entities.get("claim_id")
            if claim_id:
                # Find specific claim
                claim = next((c for c in patient_data["claims"] if c["claim_id"] == claim_id), None)
                if claim:
                    context = f"Claim {claim_id} status: {claim['status']}. "
                    context += f"Applied on: {claim['application_date']}. "
                    if claim.get("expected_resolution"):
                        context += f"Expected resolution by: {claim['expected_resolution']}. "
            else:
                # Include all claims
                claims_info = [f"Claim {c['claim_id']}: {c['status']}" for c in patient_data["claims"]]
                context = "Patient has the following claims: " + ", ".join(claims_info)
        
        # Add more context generation logic for other intents
        
        return context
        
    def process_query(self, query: str, patient_id: str) -> str:
        """Process user query and generate response"""
        # Get patient data
        patient_data = self._fetch_patient_data(patient_id)
        
        # Detect intent and extract entities
        intent = self._detect_intent(query)
        entities = self._extract_entities(query)
        
        # Generate response
        response = self._generate_response(intent, entities, patient_data, query)
        
        return response

# Usage example
chatbot = PatientChatbot(
    mongodb_uri="mongodb://localhost:27017/",
    gemini_api_key=creds.api_key
)

response = chatbot.process_query(
    "What's the status of my claim?", 
    patient_id="67f5dfefa479da68952deab6"
)
print(response)

I understand you'd like to know the status of your claim. Unfortunately, I don't have access to your specific insurance claim information at this time. To find out about your claim's status, please contact your insurance provider directly; their contact information is usually on your insurance card.



Term Explainer:

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pymongo import MongoClient
import google.generativeai as genai
import re
import creds

# MongoDB Configuration (ensure these match your main script)
MONGO_CONFIG = {
    "host": "127.0.0.1",
    "port": 27017,
    "db_name": "ClaimDetails",
}

# Gemini API Configuration
GEMINI_API_KEY = "YOUR_API_KEY"  # Replace with your Gemini API key
genai.configure(api_key=creds.api_key)

# Chatbot Interface
chat_input = widgets.Text(description="You:")
chat_output = widgets.Output()
send_button = widgets.Button(description="Send")
chat_output.layout.overflow = 'auto' # Enable vertical scrolling

def get_gemini_explanation(prompt):
    """Generates an explanation using the Gemini API."""
    model = genai.GenerativeModel('gemini-2.0-flash')
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Gemini API Error: {e}"

def get_data_from_mongodb(insurance_id, field):
    """Retrieves data from MongoDB based on Insurance ID and field."""
    try:
        client = MongoClient(MONGO_CONFIG["host"], MONGO_CONFIG["port"])
        db = client[MONGO_CONFIG["db_name"]]
        patient_details = db.patient_details

        patient_data = patient_details.find_one({"insurance_id": insurance_id})
        if patient_data and patient_data.get(field):
            return patient_data[field]
        else:
            return "Data not found in database."

    except Exception as e:
        return f"MongoDB Error: {e}"

def handle_query(button):
    """Handles user queries and generates responses."""
    query = chat_input.value.strip().lower() #added lower and strip
    clear_output(wait=True)
    with chat_output:
        print(f"You: {query}")

        # Check for insurance-related queries
        if any(keyword in query for keyword in ["insurance", "policy", "claim", "deductible", "premium"]):
            prompt = f"Explain the insurance term '{query}'. Provide a clear and concise explanation."
            response = get_gemini_explanation(prompt)
            print(f"Chatbot: {response}")

        # Check for medical-related queries
        elif any(keyword in query for keyword in ["medical", "disease", "medicine", "hypertension", "diabetes", "appendicitis"]):
            prompt = f"Explain the medical term '{query}'. Provide a clear and concise explanation."
            response = get_gemini_explanation(prompt)
            print(f"Chatbot: {response}")

        # Check for MongoDB data queries
        elif any(keyword in query for keyword in ["data", "information", "details", "name", "father", "aadhar", "gender", "blood", "address", "hospital", "phone", "disease", "medicines", "bed", "amount", "charges"]):
            insurance_id_match = re.search(r'\b[A-Za-z0-9-]+\b', query)
            if insurance_id_match:
                insurance_id = insurance_id_match.group(0)
                field_match = re.search(r'(name|father|aadhar|gender|blood|address|hospital|phone|disease|medicines|bed|amount|charges)', query, re.IGNORECASE)
                if field_match:
                    field = field_match.group(0).lower().replace("'", "")
                    response = get_data_from_mongodb(insurance_id, field)
                    print(f"Chatbot: {response}")
                else:
                    print("Chatbot: Please specify a field (name, address, etc.).")
            else:
                print("Chatbot: Please provide an Insurance ID.")
        else:
            print("Chatbot: I can help explain medical and insurance terms, and retrieve data from the extracted claims. Please specify what you are looking for.")

    chat_input.value = ""

send_button.on_click(handle_query)
display(chat_input, send_button, chat_output)

Text(value='', description='You:')

Button(description='Send', style=ButtonStyle())

Output(layout=Layout(overflow='auto'))

Using gradio:

In [ ]:
import gradio as gr
from pymongo import MongoClient
import google.generativeai as genai
import re
import creds

# MongoDB Configuration (ensure these match your main script)
MONGO_CONFIG = {
    "host": "127.0.0.1",
    "port": 27017,
    "db_name": "ClaimDetails",
}

# Gemini API Configuration
GEMINI_API_KEY = "YOUR_API_KEY"  # Replace with your Gemini API key
genai.configure(api_key=creds.api_key)

def get_gemini_explanation(prompt):
    """Generates an explanation using the Gemini API."""
    model = genai.GenerativeModel('gemini-2.0-flash')
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Gemini API Error: {e}"

def get_data_from_mongodb(insurance_id, field):
    """Retrieves data from MongoDB based on Insurance ID and field."""
    try:
        client = MongoClient(MONGO_CONFIG["host"], MONGO_CONFIG["port"])
        db = client[MONGO_CONFIG["db_name"]]
        patient_details = db.patient_details
        patient_data = patient_details.find_one({"insurance_id": insurance_id})
        
        if patient_data and patient_data.get(field):
            return patient_data[field]
        else:
            return "Data not found in database."
    except Exception as e:
        return f"MongoDB Error: {e}"

def handle_query(query, history):
    """Handles user queries and generates responses."""
    query = query.strip().lower()
    
    # Check for insurance-related queries
    if any(keyword in query for keyword in ["insurance", "policy", "claim", "deductible", "premium"]):
        prompt = f"Explain the insurance term '{query}'. Provide a clear and concise explanation."
        response = get_gemini_explanation(prompt)
    
    # Check for medical-related queries
    elif any(keyword in query for keyword in ["medical", "disease", "medicine", "hypertension", "diabetes", "appendicitis"]):
        prompt = f"Explain the medical term '{query}'. Provide a clear and concise explanation."
        response = get_gemini_explanation(prompt)
    
    # Check for MongoDB data queries
    elif any(keyword in query for keyword in ["data", "information", "details", "name", "father", "aadhar", "gender", "blood", "address", "hospital", "phone", "disease", "medicines", "bed", "amount", "charges"]):
        insurance_id_match = re.search(r'\b[A-Za-z0-9-]+\b', query)
        
        if insurance_id_match:
            insurance_id = insurance_id_match.group(0)
            field_match = re.search(r'(name|father|aadhar|gender|blood|address|hospital|phone|disease|medicines|bed|amount|charges)', query, re.IGNORECASE)
            
            if field_match:
                field = field_match.group(0).lower().replace("'", "")
                response = get_data_from_mongodb(insurance_id, field)
            else:
                response = "Please specify a field (name, address, etc.)."
        else:
            response = "Please provide an Insurance ID."
    
    else:
        response = "I can help explain medical and insurance terms, and retrieve data from the extracted claims. Please specify what you are looking for."
    
    history.append((query, response))
    return history, ""

# Create Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Insurance and Medical Chatbot")
    
    chatbot = gr.Chatbot(label="Conversation")
    msg = gr.Textbox(label="You:", placeholder="Type your message here...")
    clear = gr.Button("Clear")
    
    msg.submit(handle_query, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot)

if __name__ == "__main__":
    demo.launch()

/opt/homebrew/Caskroom/miniconda/base/envs/jupyter_env/lib/python3.11/site-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/jupyter_env/lib/python3.11/site-packages/gradio/routes.py", line 585, in main
    resp = templates.TemplateResponse(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/jupyter_env/lib/python3.11/site-packages/starlette/templating.py", line 208, in TemplateResponse
    template = self.get_template(name)
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/j

* Running on local URL:  http://127.0.0.1:7860


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/jupyter_env/lib/python3.11/site-packages/gradio/routes.py", line 585, in main
    resp = templates.TemplateResponse(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/jupyter_env/lib/python3.11/site-packages/starlette/templating.py", line 208, in TemplateResponse
    template = self.get_template(name)
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/jupyter_env/lib/python3.11/site-packages/starlette/templating.py", line 131, in get_template
    return self.env.get_template(name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/jupyter_env/lib/python3.11/site-packages/jinja2/environment.py", line 1016, in get_template
    return self._load_template(name, globals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/mini

ValueError: When localhost is not accessible, a shareable link must be created. Please set share=True or check your proxy settings to allow access to localhost.